In [1]:
import sys
import os
import datetime
import time
import math

import numpy as np
import skimage as sk
from skimage import io
import skimage.measure
import skimage.morphology
import sklearn.metrics
import pandas as pd
import h5py

%matplotlib inline
from matplotlib import pyplot as plt

import data.datasets.I3 as I3
import data.datasets.LW4 as LW4

selem = skimage.morphology.disk(3)

# Our Methods

In [2]:
results_df = pd.read_csv("results/exp.csv", sep=",")

In [3]:
results_df['modelname'] = results_df['modelname'].map(lambda x: str(x)[:-11])

In [4]:
results_df = results_df.replace("I_2021_256_1_1_0", "00_I3_UNET")
results_df = results_df.replace("I_2021_256_1_2_0", "01_I3_EUNET")
#results_df = results_df.replace("I_2021_256_1_3_0", "02_I3_EUNET_SM")

results_df = results_df.replace("I_2021_256_1_1_1_3200_650", "03_I3_M_UNET")
results_df = results_df.replace("I_2021_256_1_2_1_3200_650", "04_I3_M_EUNET")
#results_df = results_df.replace("I_2021_256_1_3_1_3200_650", "05_I3_M_EUNET_SM")

results_df = results_df.replace("I_2021_256_2_1_0", "10_LW4_UNET")
results_df = results_df.replace("I_2021_256_2_2_0", "11_LW4_EUNET")
#results_df = results_df.replace("I_2021_256_2_3_0", "12_LW4_EUNET_SM")

results_df = results_df.replace("I_2021_256_2_1_1_3200_650", "13_M_LW4_UNET")
results_df = results_df.replace("I_2021_256_2_2_1_3200_650", "14_M_LW4_EUNET")
#results_df = results_df.replace("I_2021_256_2_3_1_3200_650", "15_M_LW4_EUNET_SM")

In [5]:
results_df.groupby(["modelname"]).count()

,F1_C1_V0,F1_C2_V0,F1_C3_V0,F1_C1_V1,F1_C2_V1,F1_C3_V1,F1_C1_V2,F1_C2_V2,F1_C3_V2,F1_C1_V3,...,F1_C3_T16,F1_C1_T17,F1_C2_T17,F1_C3_T17,F1_C1_T18,F1_C2_T18,F1_C3_T18,F1_C1_T19,F1_C2_T19,F1_C3_T19
modelname,,,,,,,,,,,,,,,,,,,,,
00_I3_UNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,0,0,0
01_I3_EUNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,0,0,0
03_I3_M_UNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,0,0,0
04_I3_M_EUNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,0,0,0
10_LW4_UNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
11_LW4_EUNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
13_M_LW4_UNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
14_M_LW4_EUNET,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15


In [6]:
results_np = results_df.to_numpy()

f1c1_valid_mean = []
f1c1_valid_std = []
f1c2_valid_mean = []
f1c2_valid_std = []
f1c3_valid_mean = []
f1c3_valid_std = []
f1c1_test_mean = []
f1c1_test_std = []
f1c2_test_mean = []
f1c2_test_std = []
f1c3_test_mean = []
f1c3_test_std = []
f1_mean_sum_valid = []
f1_mean_sum_test = []
    
for model in range(results_np.shape[0]):
    # F1 Valid mean/std
    f1c1 = []
    f1c2 = []
    f1c3 = []
    
    for i in range(0,20):
        if results_np[model,3*i+1] + results_np[model,3*i+2] + results_np[model,3*i+3] > 0:
            f1c1.append(results_np[model,3*i+1]) 
            f1c2.append(results_np[model,3*i+2])
            f1c3.append(results_np[model,3*i+3])
            
    f1c1 = np.array(f1c1)
    f1c2 = np.array(f1c2)
    f1c3 = np.array(f1c3)
    
    f1c1_valid_mean.append(f1c1.mean())
    f1c1_valid_std.append(f1c1.std())
    f1c2_valid_mean.append(f1c2.mean())
    f1c2_valid_std.append(f1c2.std())
    f1c3_valid_mean.append(f1c3.mean())
    f1c3_valid_std.append(f1c3.std())
    f1_mean_sum_valid.append(f1c1.mean()+f1c2.mean()+f1c3.mean())
    
    # F1 Test mean/std
    f1c1 = []
    f1c2 = []
    f1c3 = []
    
    for i in range(0,20):
        if results_np[model,60+3*i+1] + results_np[model,60+3*i+2] + results_np[model,60+3*i+3] > 0:
            f1c1.append(results_np[model,60+3*i+1]) 
            f1c2.append(results_np[model,60+3*i+2])
            f1c3.append(results_np[model,60+3*i+3])
        
    f1c1 = np.array(f1c1)
    f1c2 = np.array(f1c2)
    f1c3 = np.array(f1c3)
    
    f1c1_test_mean.append(f1c1.mean())
    f1c1_test_std.append(f1c1.std())
    f1c2_test_mean.append(f1c2.mean())
    f1c2_test_std.append(f1c2.std())
    f1c3_test_mean.append(f1c3.mean())
    f1c3_test_std.append(f1c3.std())
    f1_mean_sum_test.append(f1c1.mean()+f1c2.mean()+f1c3.mean())
    
results_df['F1_C1_V_MEAN'] = f1c1_valid_mean
results_df['F1_C1_V_STD'] = f1c1_valid_std
results_df['F1_C2_V_MEAN'] = f1c2_valid_mean
results_df['F1_C2_V_STD'] = f1c2_valid_std
results_df['F1_C3_V_MEAN'] = f1c3_valid_mean
results_df['F1_C3_V_STD'] = f1c3_valid_std
results_df['F1_C1_T_MEAN'] = f1c1_test_mean
results_df['F1_C1_T_STD'] = f1c1_test_std
results_df['F1_C2_T_MEAN'] = f1c2_test_mean
results_df['F1_C2_T_STD'] = f1c2_test_std
results_df['F1_C3_T_MEAN'] = f1c3_test_mean
results_df['F1_C3_T_STD'] = f1c3_test_std

results_df['F1_MEAN_SUM_V'] = f1_mean_sum_valid
results_df['F1_MEAN_SUM_T'] = f1_mean_sum_test

In [7]:
results_df.nlargest(10,'F1_MEAN_SUM_T')

,modelname,F1_C1_V0,F1_C2_V0,F1_C3_V0,F1_C1_V1,F1_C2_V1,F1_C3_V1,F1_C1_V2,F1_C2_V2,F1_C3_V2,...,F1_C3_V_MEAN,F1_C3_V_STD,F1_C1_T_MEAN,F1_C1_T_STD,F1_C2_T_MEAN,F1_C2_T_STD,F1_C3_T_MEAN,F1_C3_T_STD,F1_MEAN_SUM_V,F1_MEAN_SUM_T
35,01_I3_EUNET,0.974346,0.839440,0.728286,0.953530,0.873599,0.730859,0.962820,0.831362,0.720150,...,0.722983,0.024889,0.958820,0.009828,0.823609,0.025065,0.759169,0.022843,2.528782,2.541598
57,04_I3_M_EUNET,0.956652,0.854500,0.730825,0.947107,0.888484,0.728560,0.967294,0.844839,0.700892,...,0.733735,0.028201,0.930655,0.026986,0.824999,0.020692,0.754761,0.024509,2.528779,2.510416
8,00_I3_UNET,0.959777,0.850625,0.724449,0.959469,0.883806,0.719008,0.971694,0.843145,0.696564,...,0.699669,0.027992,0.945940,0.014777,0.821657,0.021427,0.738734,0.029236,2.504636,2.506331
16,03_I3_M_UNET,0.970464,0.847422,0.728471,0.962892,0.880243,0.710274,0.972140,0.838882,0.677010,...,0.707424,0.028233,0.953074,0.012831,0.813956,0.021106,0.737272,0.026992,2.512764,2.504302
24,03_I3_M_UNET,0.977370,0.854480,0.727609,0.957304,0.881790,0.712474,0.973091,0.823261,0.697527,...,0.710048,0.023532,0.945303,0.015684,0.812362,0.023709,0.743319,0.028132,2.509916,2.500984
51,04_I3_M_EUNET,0.969921,0.845703,0.734057,0.966436,0.875535,0.711127,0.973288,0.849067,0.718655,...,0.720027,0.021845,0.954865,0.013677,0.799685,0.031945,0.741680,0.024035,2.527624,2.496230
17,03_I3_M_UNET,0.972789,0.845938,0.690158,0.959161,0.874893,0.677100,0.963642,0.828220,0.686325,...,0.693253,0.022070,0.940484,0.025180,0.819559,0.025069,0.728393,0.021155,2.486557,2.488437
23,03_I3_M_UNET,0.954481,0.816057,0.715286,0.961792,0.855947,0.704677,0.971467,0.827994,0.683338,...,0.701096,0.026680,0.953184,0.009793,0.792758,0.028204,0.740204,0.024545,2.484627,2.486145
27,03_I3_M_UNET,0.934963,0.840596,0.704197,0.944378,0.879025,0.698569,0.956676,0.848391,0.674082,...,0.699584,0.025638,0.911063,0.038421,0.827589,0.022242,0.746729,0.022336,2.480322,2.485381
14,00_I3_UNET,0.969296,0.836998,0.687420,0.971315,0.881137,0.690276,0.971354,0.831606,0.676999,...,0.687970,0.026419,0.949978,0.018177,0.808819,0.022552,0.724880,0.022947,2.479657,2.483677


In [8]:
for cl in ['F1_C1_T_MEAN', 'F1_C2_T_MEAN', 'F1_C3_T_MEAN']:
    for modelname in sorted(results_df.modelname.unique()):
        print(
            modelname, cl, ' \t',
            results_df.loc[results_df['modelname'] == modelname].nlargest(10,'F1_MEAN_SUM_V')[[cl]].to_numpy().mean(),
            results_df.loc[results_df['modelname'] == modelname].nlargest(10,'F1_MEAN_SUM_V')[[cl]].to_numpy().std(),
            results_df.loc[results_df['modelname'] == modelname].nlargest(10,'F1_MEAN_SUM_V')[[cl]].to_numpy().max()
            #results_df.loc[results_df['modelname'] == modelname][[cl]].to_numpy().mean(),
            #results_df.loc[results_df['modelname'] == modelname][[cl]].to_numpy().std()
        )

00_I3_UNET F1_C1_T_MEAN  	 0.9379011593600808 0.009907178706630715 0.9507617340300724
01_I3_EUNET F1_C1_T_MEAN  	 0.9515793857196592 0.008192720704163593 0.9589013844052126
03_I3_M_UNET F1_C1_T_MEAN  	 0.9391373112575732 0.013267370377149706 0.9531840934556456
04_I3_M_EUNET F1_C1_T_MEAN  	 0.9477952074251569 0.011831469456374135 0.9591685460709933
10_LW4_UNET F1_C1_T_MEAN  	 0.9553274206472345 0.0027545690318627864 0.9582627925633546
11_LW4_EUNET F1_C1_T_MEAN  	 0.9575729314536439 0.0013269103903690129 0.959518643697038
13_M_LW4_UNET F1_C1_T_MEAN  	 0.9505278589356134 0.0028571935756098177 0.9570564022195054
14_M_LW4_EUNET F1_C1_T_MEAN  	 0.9557025527328268 0.0022414769052213554 0.958852448554595
00_I3_UNET F1_C2_T_MEAN  	 0.8066406174899077 0.012610455790889529 0.8216567719247203
01_I3_EUNET F1_C2_T_MEAN  	 0.7647469571458647 0.03604879055882932 0.8236094506691051
03_I3_M_UNET F1_C2_T_MEAN  	 0.8004994225767593 0.019924957678039673 0.8275892176745565
04_I3_M_EUNET F1_C2_T_MEAN  	 0.75

# Selected Model (on valid)

In [9]:
# results_df = pd.read_csv("results/exp.csv", sep=",")

exec cell to compute mean

In [10]:
# results_df.nlargest(60,'F1_MEAN_SUM_V')[["modelname", "F1_MEAN_SUM_V", "F1_C1_V_MEAN", "F1_C2_V_MEAN", "F1_C3_V_MEAN"]]

* selected model for I3 :  I_2021_256_1_2_1_3200_650_1627262476
* selected model for LW4 : I_2021_256_2_2_0_0926302336

In [11]:
def cmp(seg1, seg2, minimum_recall_tp=0.80):
    # results
    tp = 0
    fn = 0
    under_detected = 0

    if not (seg1.shape == seg2.shape):
        print("ERROR: segmentation shape not equal")
        return 1

    # The argument 'neighbors' is deprecated, use 'connectivity' instead. For neighbors=8, use connectivity=2.
    seg1_labels = skimage.measure.label(seg1, connectivity=2)
    seg1_n_cc = seg1_labels.max()

    for cc_label in range(1, seg1_n_cc + 1):
        # the current connected component
        current_cc = ((seg1_labels == cc_label) * 1).astype(np.uint8)
        # the intersection
        intersected_cc = current_cc * seg2
        
        recall = np.sum(current_cc * intersected_cc) / np.sum(current_cc)
        
        if np.sum(intersected_cc) == 0 or recall < 0.10:
            fn += 1
        elif recall > minimum_recall_tp:
            tp += 1
        else:
            under_detected += 1
    
    return (seg1_n_cc, tp, fn, under_detected)

def print_cmp(res):
    cc, tp, fn, ud = res
    print("(/"+str(cc)+") & "+str(round((tp/cc)*100, 1))+"\\% ("+str(tp)+") & "+str(round((fn/cc)*100, 1))+"\\% ("+str(fn)+") & "+str(round((ud/cc)*100, 1))+"\\% ("+str(ud)+") \\\\")

In [12]:
pred_i3_labels = np.load("results/I_2021_256_1_2_1_3200_650_1627262476.npy")
pred_lw4_labels = np.load("results/I_2021_256_2_2_0_0926302336.npy")

In [13]:
pred_valid_i3_labels = pred_i3_labels[0:20]
pred_test_i3_labels = pred_i3_labels[20:]
pred_valid_lw4_labels = pred_lw4_labels[0:20]
pred_test_lw4_labels = pred_lw4_labels[20:]

In [14]:
result = np.array([0, 0, 0, 0])
for z in range(pred_test_i3_labels.shape[0]):
    result = result + cmp(I3.test_i3_label_1[z], pred_test_i3_labels[z]==1, 0.80)
print_cmp(result)

(/193) & 92.7\% (179) & 4.7\% (9) & 2.6\% (5) \\


In [15]:
result = np.array([0, 0, 0, 0])
for z in range(pred_test_i3_labels.shape[0]):
    result = result + cmp(I3.test_i3_label_3[z], pred_test_i3_labels[z]==3, 0.80)
print_cmp(result)

(/1034) & 46.9\% (485) & 19.1\% (198) & 33.9\% (351) \\


In [16]:
result = np.array([0, 0, 0, 0])
for z in range(pred_test_lw4_labels.shape[0]):
    result = result + cmp(LW4.test_lw4_label_1[z], pred_test_lw4_labels[z]==1, 0.80)
print_cmp(result)

(/363) & 89.0\% (323) & 6.1\% (22) & 5.0\% (18) \\


In [17]:
result = np.array([0, 0, 0, 0])
for z in range(pred_test_lw4_labels.shape[0]):
    result = result + cmp(LW4.test_lw4_label_3[z], pred_test_lw4_labels[z]==3, 0.80)
print_cmp(result)

(/1468) & 56.1\% (824) & 21.5\% (316) & 22.3\% (328) \\


# Second Expert

In [18]:
i3_image = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/I3/i3.tif"))

lw4_image = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/LW4/LW4-600.tif"))

i3_label_1 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/I3/Labels_i3-mitos_1-500.tif"))
i3_label_2 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/I3/Labels_i3_MembraneCellule_1-250.tif"))
i3_label_3 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/I3/Labels_i3_Reticulum_172-251.tif"))

lw4_label_1 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/LW4/Labels_LW4-600_All-Step40_mito.tif"))
lw4_label_2 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/LW4/Labels_LW4-600_All-Step40_cell.tif"))
lw4_label_3 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/LW4/Labels_LW4-600_1-40_81-120_Reti.tif"))

In [19]:
e2_i3_label_1 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/EXPERT_2/Labels_i3_Mito_240_E2.tif"))
e2_i3_label_2 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/EXPERT_2/Labels_i3_Cell_240_E2.tif"))
e2_i3_label_3 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/EXPERT_2/Labels_i3_Reti_240_E2.tif"))

e2_lw4_label_1 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/EXPERT_2/Labels_LW4-600_110_Mito_E2.tif"))
e2_lw4_label_2 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/EXPERT_2/Labels_LW4-600_110_Cell_E2.tif"))
e2_lw4_label_3 = np.array(
    sk.io.imread("/home/cyril/Documents/Data/MEYER_ISBI_2021/EXPERT_2/Labels_LW4-600_110_Reti_E2.tif"))

In [20]:
print(i3_label_1[239].min(), i3_label_1[239].max(), i3_label_1[239].sum())
print(i3_label_2[239].min(), i3_label_2[239].max(), i3_label_2[239].sum())
print(i3_label_3[239].min(), i3_label_3[239].max(), i3_label_3[239].sum())
print(lw4_label_1[109].min(), lw4_label_1[109].max(), lw4_label_1[109].sum())
print(lw4_label_2[109].min(), lw4_label_2[109].max(), lw4_label_2[109].sum())
print(lw4_label_3[109].min(), lw4_label_3[109].max(), lw4_label_3[109].sum())

0 1 126680
0 1 13090
0 1 86729
0 1 112878
0 1 48213
0 1 76784


In [21]:
print(e2_i3_label_1[239].min(), e2_i3_label_1[239].max(), e2_i3_label_1[239].sum())
print(e2_i3_label_2[239].min(), e2_i3_label_2[239].max(), e2_i3_label_2[239].sum())
print(e2_i3_label_3[239].min(), e2_i3_label_3[239].max(), e2_i3_label_3[239].sum())
print(e2_lw4_label_1[109].min(), e2_lw4_label_1[109].max(), e2_lw4_label_1[109].sum())
print(e2_lw4_label_2[109].min(), e2_lw4_label_2[109].max(), e2_lw4_label_2[109].sum())
print(e2_lw4_label_3[109].min(), e2_lw4_label_3[109].max(), e2_lw4_label_3[109].sum())

0 1 115609
0 1 13997
0 1 78499
0 1 120116
0 1 48700
0 1 62475


In [22]:
print("F1 I3 C1", sklearn.metrics.f1_score(i3_label_1[239].flatten(), e2_i3_label_1[239].flatten()))
print("F1 I3 C2", sklearn.metrics.f1_score(i3_label_2[239].flatten(), e2_i3_label_2[239].flatten()))
print("F1 I3 C3", sklearn.metrics.f1_score(i3_label_3[239].flatten(), e2_i3_label_3[239].flatten()))
print("F1 LW4 C1", sklearn.metrics.f1_score(lw4_label_1[109].flatten(), e2_lw4_label_1[109].flatten()))
print("F1 LW4 C2", sklearn.metrics.f1_score(lw4_label_2[109].flatten(), e2_lw4_label_2[109].flatten()))
print("F1 LW4 C3", sklearn.metrics.f1_score(lw4_label_3[109].flatten(), e2_lw4_label_3[109].flatten()))

F1 I3 C1 0.9418834532314715
F1 I3 C2 0.7510613947650164
F1 I3 C3 0.7407945384559519
F1 LW4 C1 0.9494321742190786
F1 LW4 C2 0.5932950171803576
F1 LW4 C3 0.7832312453773185


In [23]:
print_cmp(cmp(i3_label_1[239],  e2_i3_label_1[239] , 0.8))
print_cmp(cmp(i3_label_3[239],  e2_i3_label_3[239] , 0.8))
print_cmp(cmp(lw4_label_1[109], e2_lw4_label_1[109], 0.8))
print_cmp(cmp(lw4_label_3[109], e2_lw4_label_3[109], 0.8))

(/10) & 100.0\% (10) & 0.0\% (0) & 0.0\% (0) \\
(/55) & 23.6\% (13) & 9.1\% (5) & 67.3\% (37) \\
(/21) & 100.0\% (21) & 0.0\% (0) & 0.0\% (0) \\
(/68) & 26.5\% (18) & 19.1\% (13) & 54.4\% (37) \\


# Ilastik

In [24]:
# MyProject3
train_ila_i3_labels_proba = h5py.File('/home/cyril/Documents/Data/ALL_ANNOTATED_STACKS/TRAIN_I3_IMAGE_NORMALIZED_F32_Probabilities.h5', 'r')['exported_data']
valid_ila_i3_labels_proba = h5py.File('/home/cyril/Documents/Data/ALL_ANNOTATED_STACKS/VALID_I3_IMAGE_NORMALIZED_F32_Probabilities.h5', 'r')['exported_data']
test_ila_i3_labels_proba = h5py.File('/home/cyril/Documents/Data/ALL_ANNOTATED_STACKS/TEST_I3_IMAGE_NORMALIZED_F32_Probabilities.h5', 'r')['exported_data']

train_ila_lw4_labels_proba = h5py.File('/home/cyril/Documents/Data/ALL_ANNOTATED_STACKS/TRAIN_LW4_IMAGE_NORMALIZED_F32_Probabilities.h5', 'r')['exported_data']
valid_ila_lw4_labels_proba = h5py.File('/home/cyril/Documents/Data/ALL_ANNOTATED_STACKS/VALID_LW4_IMAGE_NORMALIZED_F32_Probabilities.h5', 'r')['exported_data']
test_ila_lw4_labels_proba = h5py.File('/home/cyril/Documents/Data/ALL_ANNOTATED_STACKS/TEST_LW4_IMAGE_NORMALIZED_F32_Probabilities.h5', 'r')['exported_data']

In [25]:
train_ila_i3_labels = np.argmax(train_ila_i3_labels_proba, axis=-1)
train_ila_i3_label_1 = (train_ila_i3_labels==0)*1
train_ila_i3_label_2 = (train_ila_i3_labels==1)*1
train_ila_i3_label_3 = (train_ila_i3_labels==2)*1

valid_ila_i3_labels = np.argmax(valid_ila_i3_labels_proba, axis=-1)
valid_ila_i3_label_1 = (valid_ila_i3_labels==0)*1
valid_ila_i3_label_2 = (valid_ila_i3_labels==1)*1
valid_ila_i3_label_3 = (valid_ila_i3_labels==2)*1

test_ila_i3_labels = np.argmax(test_ila_i3_labels_proba, axis=-1)
test_ila_i3_label_1 = (test_ila_i3_labels==0)*1
test_ila_i3_label_2 = (test_ila_i3_labels==1)*1
test_ila_i3_label_3 = (test_ila_i3_labels==2)*1

train_ila_lw4_labels = np.argmax(train_ila_lw4_labels_proba, axis=-1)
train_ila_lw4_label_1 = (train_ila_lw4_labels==0)*1
train_ila_lw4_label_2 = (train_ila_lw4_labels==1)*1
train_ila_lw4_label_3 = (train_ila_lw4_labels==2)*1

valid_ila_lw4_labels = np.argmax(valid_ila_lw4_labels_proba, axis=-1)
valid_ila_lw4_label_1 = (valid_ila_lw4_labels==0)*1
valid_ila_lw4_label_2 = (valid_ila_lw4_labels==1)*1
valid_ila_lw4_label_3 = (valid_ila_lw4_labels==2)*1

test_ila_lw4_labels = np.argmax(test_ila_lw4_labels_proba, axis=-1)
test_ila_lw4_label_1 = (test_ila_lw4_labels==0)*1
test_ila_lw4_label_2 = (test_ila_lw4_labels==1)*1
test_ila_lw4_label_3 = (test_ila_lw4_labels==2)*1

In [26]:
print("F1 I3 C1", sklearn.metrics.f1_score(I3.train_i3_label_1[0].flatten(), train_ila_i3_label_1[0].flatten()))
print("F1 I3 C2", sklearn.metrics.f1_score(I3.train_i3_label_2[0].flatten(), train_ila_i3_label_2[0].flatten()))
print("F1 I3 C3", sklearn.metrics.f1_score(I3.train_i3_label_3[0].flatten(), train_ila_i3_label_3[0].flatten()))
print("F1 I3 C1", sklearn.metrics.f1_score(I3.train_i3_label_1[1].flatten(), train_ila_i3_label_1[1].flatten()))
print("F1 I3 C2", sklearn.metrics.f1_score(I3.train_i3_label_2[1].flatten(), train_ila_i3_label_2[1].flatten()))
print("F1 I3 C3", sklearn.metrics.f1_score(I3.train_i3_label_3[1].flatten(), train_ila_i3_label_3[1].flatten()))

F1 I3 C1 0.9999820937214619
F1 I3 C2 0.9999575785856699
F1 I3 C3 0.9999871389253285
F1 I3 C1 0.17159361196058442
F1 I3 C2 0.743839646929018
F1 I3 C3 0.14967544039659997


In [27]:
print("F1 LW4 C1", sklearn.metrics.f1_score(LW4.train_lw4_label_1[0].flatten(), train_ila_lw4_label_1[0].flatten()))
print("F1 LW4 C2", sklearn.metrics.f1_score(LW4.train_lw4_label_2[0].flatten(), train_ila_lw4_label_2[0].flatten()))
print("F1 LW4 C3", sklearn.metrics.f1_score(LW4.train_lw4_label_3[0].flatten(), train_ila_lw4_label_3[0].flatten()))
print("F1 LW4 C1", sklearn.metrics.f1_score(LW4.train_lw4_label_1[1].flatten(), train_ila_lw4_label_1[1].flatten()))
print("F1 LW4 C2", sklearn.metrics.f1_score(LW4.train_lw4_label_2[1].flatten(), train_ila_lw4_label_2[1].flatten()))
print("F1 LW4 C3", sklearn.metrics.f1_score(LW4.train_lw4_label_3[1].flatten(), train_ila_lw4_label_3[1].flatten()))

F1 LW4 C1 0.9999907721836705
F1 LW4 C2 0.9999727285123404
F1 LW4 C3 0.9999354241556708
F1 LW4 C1 0.8423694873506379
F1 LW4 C2 0.5393061854008885
F1 LW4 C3 0.3542957374321149


In [28]:
print("F1 I3 C1 VALID", sklearn.metrics.f1_score(I3.valid_i3_label_1.flatten(), valid_ila_i3_label_1.flatten()))
print("F1 I3 C2 VALID", sklearn.metrics.f1_score(I3.valid_i3_label_2.flatten(), valid_ila_i3_label_2.flatten()))
print("F1 I3 C3 VALID", sklearn.metrics.f1_score(I3.valid_i3_label_3.flatten(), valid_ila_i3_label_3.flatten()))
print("F1 I3 C1", sklearn.metrics.f1_score(I3.test_i3_label_1.flatten(), test_ila_i3_label_1.flatten()))
print("F1 I3 C2", sklearn.metrics.f1_score(I3.test_i3_label_2.flatten(), test_ila_i3_label_2.flatten()))
print("F1 I3 C3", sklearn.metrics.f1_score(I3.test_i3_label_3.flatten(), test_ila_i3_label_3.flatten()))

F1 I3 C1 VALID 0.16804258810983105
F1 I3 C2 VALID 0.6931411958249455
F1 I3 C3 VALID 0.1153067771155303
F1 I3 C1 0.1858275657998928
F1 I3 C2 0.6726905156323444
F1 I3 C3 0.11266295922665764


In [29]:
print("F1 LW4 C1 VALID", sklearn.metrics.f1_score(LW4.valid_lw4_label_1.flatten(), valid_ila_lw4_label_1.flatten()))
print("F1 LW4 C2 VALID", sklearn.metrics.f1_score(LW4.valid_lw4_label_2.flatten(), valid_ila_lw4_label_2.flatten()))
print("F1 LW4 C3 VALID", sklearn.metrics.f1_score(LW4.valid_lw4_label_3.flatten(), valid_ila_lw4_label_3.flatten()))
print("F1 LW4 C1", sklearn.metrics.f1_score(LW4.test_lw4_label_1.flatten(), test_ila_lw4_label_1.flatten()))
print("F1 LW4 C2", sklearn.metrics.f1_score(LW4.test_lw4_label_2.flatten(), test_ila_lw4_label_2.flatten()))
print("F1 LW4 C3", sklearn.metrics.f1_score(LW4.test_lw4_label_3.flatten(), test_ila_lw4_label_3.flatten()))

F1 LW4 C1 VALID 0.8714802164791043
F1 LW4 C2 VALID 0.49942066608307584
F1 LW4 C3 VALID 0.22487924225603723
F1 LW4 C1 0.8724314710389329
F1 LW4 C2 0.5206576300874053
F1 LW4 C3 0.2166315532495367


In [30]:
result = np.array([0, 0, 0, 0])
for z in range(test_ila_i3_label_1.shape[0]):
    result = result + cmp(I3.test_i3_label_1[z], test_ila_i3_label_1[z], 0.80)
print_cmp(result)

(/193) & 0.0\% (0) & 49.2\% (95) & 50.8\% (98) \\


In [31]:
result = np.array([0, 0, 0, 0])
for z in range(test_ila_i3_label_3.shape[0]):
    result = result + cmp(I3.test_i3_label_3[z], test_ila_i3_label_3[z], 0.80)
print_cmp(result)

(/1034) & 0.0\% (0) & 82.2\% (850) & 17.8\% (184) \\


In [32]:
result = np.array([0, 0, 0, 0])
for z in range(test_ila_lw4_label_1.shape[0]):
    result = result + cmp(LW4.test_lw4_label_1[z], test_ila_lw4_label_1[z], 0.80)
print_cmp(result)

(/363) & 62.8\% (228) & 4.7\% (17) & 32.5\% (118) \\


In [33]:
result = np.array([0, 0, 0, 0])
for z in range(test_ila_lw4_label_3.shape[0]):
    result = result + cmp(LW4.test_lw4_label_3[z], test_ila_lw4_label_3[z], 0.80)
print_cmp(result)

(/1468) & 0.0\% (0) & 60.6\% (889) & 39.4\% (579) \\
